# Fine-Tuning the Text to Image Model

Let's teach our text to image generator what my dog looks like.

![redhat dog](https://rhods-public.s3.amazonaws.com/sample-data/images/redhat-dog-small.jpg)

### Check GPU

**It's recommended that you shut down any other notebook kernels.**

This fine tuning process uses a lot of video memory.   Here, we'll check on how much we have available.

In [ ]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

## Install Requirements

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip list | grep -E "boto|grpcio|pandas|torch|torchvision|diffusers|transformers|accelerate|flash-attn|ftfy|xformers|protobuf"

## Settings

Here we set up all the options for training.  Most are environment variables which will allow us to override values from pipelines and run this notebook with different setting such as the base model or number of training steps and learning rate.

In [ ]:
import os
from datetime import datetime

date = datetime.now()
date_string = date.strftime("%Y%m%d%H%M%S")

VERSION = os.environ.get("VERSION", f"notebook-output")
MODEL_NAME = os.environ.get("MODEL_NAME", "runwayml/stable-diffusion-v1-5")
OUTPUT_DIR = os.path.join(os.getcwd(), f"{VERSION}/stable_diffusion_weights/redhat-dog")
DATA_DIR = os.path.join(os.getcwd(), f"{VERSION}/data")
INSTANCE_DATA_URL = os.environ.get("INSTANCE_DATA_URL", "https://rhods-public.s3.amazonaws.com/sample-data/images/redhat-dog.tar.gz")
INSTANCE_DIR = os.path.join(DATA_DIR, "instance_dir")
CLASS_DIR = os.path.join(DATA_DIR, "class_dir")
INSTANCE_PROMPT = os.environ.get("INSTANCE_PROMPT", "photo of a rhteddy dog")
CLASS_PROMPT = os.environ.get("CLASS_PROMPT", "a photo of dog")

NUM_CLASS_IMAGES = int(os.environ.get("NUM_CLASS_IMAGES", "200"))
MAX_TRAIN_STEPS = int(os.environ.get("MAX_TRAIN_STEPS", "800"))

GPU_ONNX_EXPORT = os.environ.get("GPU_ONNX_EXPORT", "TRUE")

ONNX_OUTPUT_DIR = os.path.join(os.getcwd(), f"{VERSION}/onnx-redhat-dog")

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(INSTANCE_DIR, exist_ok=True)

print(f"Weights will be saved at {OUTPUT_DIR}")
print(f"It will be based on the model {MODEL_NAME}")
print(f"Training data located in downloaded from {INSTANCE_DATA_URL}")
print(f"We're going to train the difference between \"{INSTANCE_PROMPT}\" and \"{CLASS_PROMPT}\"")


## Training

### Get Training Data


In [ ]:
import sys
import os
import tarfile
import urllib

url = INSTANCE_DATA_URL
output = f"instance-images.tar.gz"
urllib.request.urlretrieve(url, output)

!tar -xzf instance-images.tar.gz -C $INSTANCE_DIR

### Set up the Training Job

In [ ]:
!accelerate config default

In [ ]:
!wget -O train_dreambooth.py https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth.py

### Start Training

Here we kick off the training job with our chosen settings.  This will take about 15 minutes depending on settings and hardware.

In [ ]:
!echo "MODEL_NAME=$MODEL_NAME"
!echo "OUTPUT_DIR=$OUTPUT_DIR"
!echo "DATA_DIR=$DATA_DIR"
!echo "INSTANCE_DIR=$INSTANCE_DIR"
!echo "CLASS_DATA_URL=$CLASS_DATA_URL"
!echo "CLASS_DIR=$CLASS_DIR"
!echo "INSTANCE_PROMPT=$INSTANCE_PROMPT"
!echo "CLASS_PROMPT=$CLASS_PROMPT"
!echo "NUM_CLASS_IMAGES=$NUM_CLASS_IMAGES"
!echo "MAX_TRAIN_STEPS=$MAX_TRAIN_STEPS"

In [ ]:
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="$INSTANCE_PROMPT" \
  --class_prompt="$CLASS_PROMPT" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=2 --gradient_checkpointing \
  --use_8bit_adam \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=$NUM_CLASS_IMAGES \
  --max_train_steps=$MAX_TRAIN_STEPS \
  --enable_xformers_memory_efficient_attention

### Examine the output

If everything went well, we should have generated model weights in the `$OUTPUT_DIR` including those for the tokenizer, vae, unet, and text encoder.  You could now load these weights and generate images with our finetuned model.  You could simply replace any references to our Huggingface model (`runwayml/stable-diffusion-v1-5`) with the path to our new weights (e.g. `./notebook-output/stable_diffusion_weights/redhat-dog`)

It's also ready for upload to Huggingface, such as [cfchase/stable-diffusion-rhteddy](https://huggingface.co/cfchase/stable-diffusion-rhteddy), where you can use those weights in another notebook or application.  To try out the custom weight from huggingface (or any other weights) with multiple images generated, try the notebook [misc-notebooks/generate_multiple.ipynb](misc-notebooks/generate_multiple.ipynb).

In [ ]:
!ls $OUTPUT_DIR

# Export to ONNX

In order to to serve the model as an API, we need to use a format the model server understands.  The [Open Neural Network Exchange (ONNX)](https://onnx.ai/) is an open format built to represent machine learning models that enables AI developers to use models with a variety of frameworks, tools, runtimes, and compilers.  Next we'll convert the model to ONNX using the process the Huggingface diffusers library recommends in its [documentation](https://huggingface.co/docs/diffusers/v0.20.0/en/optimization/onnx).


### Install Dependencies

A new key dependency will be the [optimum](https://github.com/huggingface/optimum) library.

In [ ]:
!pip uninstall optimum["onnxruntime"]  optimum["onnxruntime-gpu"] -y

In [ ]:
import torch

if GPU_ONNX_EXPORT.upper == "TRUE" and torch.cuda.is_available():
    !pip install optimum["onnxruntime-gpu"]
else:
    !pip install optimum["onnxruntime"]

By loading the model into an `ORTStableDiffusionPipeline` and `save_pretrained`, we will end up with 4 separate ONNX models, `text_encoder`, `unet`, `vae_decoder`, and `vae_encoder`.

Also, let's see what a generated image of `rhteddy` looks like.  Before, the model had no idea, but now, it should generate a picture of Teddy.

In [ ]:
from optimum.onnxruntime import ORTStableDiffusionPipeline
import torch

model_id = OUTPUT_DIR
pipeline = ORTStableDiffusionPipeline.from_pretrained(model_id, export=True)

device = torch.device("cuda" if GPU_ONNX_EXPORT.upper == "TRUE" and torch.cuda.is_available() else "cpu")
print(device)
pipeline.to(device)
prompt = "photo of a rhteddy dog"
image = pipeline(prompt).images[0]
pipeline.save_pretrained(ONNX_OUTPUT_DIR)
image

### Did it look like Teddy?

## Save to S3

Now we have our model in a portable ONNX format, however, it's not doing much good in this notebook.  We need to push the model to our connected storage location.  Then we can use it in another notebook or serve the models for use within an application.

Note:  This requires a data connection to an S3 compatible bucket.  As part of the setup for this project, you added the [setup-s3.yaml](https://github.com/cfchase/text-to-image-demo/blob/main/setup/setup-s3.yaml) which created a local s3 bucket and data connections.

In [ ]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory_to_s3(local_directory, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)


In [ ]:
upload_directory_to_s3(ONNX_OUTPUT_DIR, f"text-to-image/{VERSION}/onnx-redhat-dog")

In [ ]:
list_objects(f"text-to-image/{VERSION}/onnx-redhat-dog")

### Remember the S3 Prefixes 

Take note of these s3 keys and prefixes.  We'll need to refer to them later when serving our model.  You'll need to provide the prefix up to each onnx file to download the onnx file and any adjacent files necessary.

For example, `path_to/unet/model.onnx` also requires `path_to/unet/model.onnx_data`, so we would use `path_to/unet` for serving.  By default, we'd use 4 paths for the 4 models:
```
text-to-image/notebook-output/onnx-redhat-dog/text_encoder
text-to-image/notebook-output/onnx-redhat-dog/unet
text-to-image/notebook-output/onnx-redhat-dog/vae_decoder
text-to-image/notebook-output/onnx-redhat-dog/vae_encoder

```

### Next

The next notebook [Notebook 3 - Remote Inference](3_remote_inference.ipynb), tests out Red Hat OpenShift Data Science Serving capabilities.  Before trying it out, you must have deployed a custom runtime for Triton, created a model server, and deployed the 4 models.

If you're interested in skipping the manual steps, you can go this project in the OpenShift console and import [serving/models.yaml](serving/models.yaml)